# Ingestion

In [44]:
import pandas as pd
from datasets import Dataset
import matplotlib.pyplot as plt
import datasets
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from typing import List, Optional  # Import List and Optional for type annotations
from langchain.docstore.document import Document as LangchainDocument
from tqdm import tqdm
import pdfplumber
import os
from datasets import Dataset
import openai
from langchain.embeddings import OpenAIEmbeddings  # Correct import path for OpenAI embeddings


In [24]:
from openai import OpenAI

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = read_token_from_file("keys/OpenAI.txt")

In [8]:
def read_token_from_file(file_path="token.txt"):
    with open(file_path, "r") as file:
        return file.read().strip()

In [9]:
file_path = [
    "C:/Users/fiori/OneDrive/Documenti/Rulebooks/"
]
COLLECTION_NAME = "transformer_sentece_splitter_2"

URL=read_token_from_file("keys/qdrant_URL.txt")
API_KEY=read_token_from_file("keys/qdrant.txt")
# Specify the folder path
folder_path = "C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks"
EMBEDDING_MODEL_NAME = "thenlper/gte-small"


In [54]:
def add_embeddings_to_qdrant(docs_processed, URL, API_KEY, OPENAI_KEY, collection_name="test_2"):
    """Create embeddings for documents and add them to Qdrant."""
    
    URL=URL 
    API_KEY=API_KEY
    
    if OPENAI_KEY:
        embedding_model = OpenAIEmbeddings(
            model="text-embedding-ada-002",  # Specify the desired OpenAI embedding model
        )

    else:
        embedding_model = HuggingFaceEmbeddings(
            model_name=EMBEDDING_MODEL_NAME,
            multi_process=True,  # Enable multiprocessing
            model_kwargs={"device": "cuda"},
            encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
        )


    vector_store = QdrantVectorStore.from_documents(
        docs_processed,
        embedding_model,
        url = URL,
        prefer_grpc=True,
        api_key=API_KEY,
        collection_name=collection_name,
        force_recreate = False
    )

In [32]:
def connect_Qdrant(URL, API_KEY):
    # Initialize Qdrant client
    qdrant_client = QdrantClient(
        url=URL, 
        api_key=API_KEY,
    )
    return qdrant_client


In [33]:
def process_pdfs(path, tokenizer):
    # Load dataset from Hugging Face
    RAW_KNOWLEDGE_BASE = []

    # List the PDF files in the folder
    pdf_files = [f for f in os.listdir(path) if f.endswith(".pdf")]
    pdf_texts = []

    for pdf_file in tqdm(pdf_files):
        # Extract text from PDF using pdfplumber
        with pdfplumber.open(os.path.join(folder_path, pdf_file)) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
        pdf_texts.append(text)

    pdf_dict = [{"game_name" : pdf_file.removesuffix(".pdf").split("_")[0], "game_id" : pdf_file.removesuffix(".pdf").split("_")[1]} for i, pdf_file in enumerate(pdf_files)]

    RAW_KNOWLEDGE_BASE = tokenizer.create_documents(
        pdf_texts,
        pdf_dict
    )

    docs_processed = tokenizer.split_documents(RAW_KNOWLEDGE_BASE)
    return docs_processed

# UNSTRUCTURED

In [12]:
from langchain_unstructured import UnstructuredLoader
from unstructured.chunking.title import chunk_by_title


loader = UnstructuredLoader(file_paths, chunk_strategy = chunk_by_title)

In [28]:
docs = loader.load()

In [29]:
docs[10].page_content

'Difficulty level:'

In [30]:
from langchain.document_loaders import UnstructuredPDFLoader #fa un pò cagare, bisognerebbe provare l'API by_title, magari si può usare per le tabelle...

loader = UnstructuredPDFLoader("C:/Users/fiori/OneDrive/Documenti/Rulebooks/The Mind Extreme.pdf", mode="elements")
docs = loader.load()

In [31]:
docs

[Document(metadata={'source': 'C:/Users/fiori/OneDrive/Documenti/Rulebooks/The Mind Extreme.pdf', 'coordinates': {'points': ((9.73945, 4.981999999999971), (9.73945, 30.98199999999997), (120.65519598000003, 30.98199999999997), (120.65519598000003, 4.981999999999971)), 'system': 'PixelSpace', 'layout_width': 257.95, 'layout_height': 334.49}, 'file_directory': 'C:/Users/fiori/OneDrive/Documenti/Rulebooks', 'filename': 'The Mind Extreme.pdf', 'languages': ['eng'], 'last_modified': '2024-09-29T13:58:58', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'Header', 'element_id': '7d22452cb6bfb2b564695db47ac3493f'}, page_content='The Mind'),
 Document(metadata={'source': 'C:/Users/fiori/OneDrive/Documenti/Rulebooks/The Mind Extreme.pdf', 'coordinates': {'points': ((194.737, 15.110000000000014), (194.737, 29.110000000000014), (217.9882, 29.110000000000014), (217.9882, 15.110000000000014)), 'system': 'PixelSpace', 'layout_width': 257.95, 'layout_height': 334.49}, 'file_directory': 'C:

# SEMANTIC SPLITTER

In [34]:
# https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb
#fare vector_Db con semantic e provare LLM
from langchain_experimental.text_splitter import SemanticChunker

In [47]:
# Initialize the embedding model
# embedding_model = HuggingFaceEmbeddings(
#     model_name=EMBEDDING_MODEL_NAME,
#     multi_process=True,  # Enable multiprocessing
#     model_kwargs={"device": "cuda"},
#     encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
# )

#nvdia embeddings + nvidia reranker with API
# from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

# embedding_model = NVIDIAEmbeddings()

embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",  # Specify the desired OpenAI embedding model
)


semnatic_splitter = SemanticChunker(
    embedding_model, breakpoint_threshold_type="percentile"
)

In [48]:
# Pass the Dataset object to the process_documents function
docs_processed = process_pdfs(folder_path, semnatic_splitter)

100%|██████████| 3/3 [00:05<00:00,  1.86s/it]


In [57]:
docs_processed

[Document(metadata={'game_name': 'The Mind Extreme'}, page_content='The Mind\nWolfgang\nWarsch\nFor professional telepaths! Players: 2-4 people Age: 8+ Duration: app.'),
 Document(metadata={'game_name': 'The Mind Extreme'}, page_content='20 min.'),
 Document(metadata={'game_name': 'The Mind Extreme'}, page_content='What‘s new about the Extreme version? The basic rules of the original „The Mind” game are the same. In the first round\n(level 1) everyone gets 1 card, in the second round (level 2) everyone gets 2 cards,\netc.'),
 Document(metadata={'game_name': 'The Mind Extreme'}, page_content='Everyone forms one team and tries to lay all the cards they receive correctly in\nthe middle of the table. There is no player order. Anyone who wants to lay a card,\nsimply lays it. Warning: There’s not just one stack in the middle of the table\nlike with the original but two at the same time! The stack with the white numbers\nis ascending (1-50), the stack with the red numbers is descending (50-1)

In [52]:
add_embeddings_to_qdrant(docs_processed, URL, API_KEY,OPENAI_KEY = os.environ["OPENAI_API_KEY"], collection_name="openai")

# SEMANTIC TRANSFORMER MODEL

In [19]:
from langchain_text_splitters.sentence_transformers import SentenceTransformersTokenTextSplitter


transformer_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=200
)

c:\Users\fiori\OneDrive\Documenti\GitHub\ChatBot_BoardGames\venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# Pass the Dataset object to the process_documents function
docs_processed = process_pdfs(folder_path, transformer_splitter)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:05<00:00,  1.99s/it]


In [11]:
len(docs_processed)

70

In [12]:
add_embeddings_to_qdrant(docs_processed, URL, API_KEY, collection_name="transformer_sentece_splitter_2")

RECURSIVE TEXT SPLITTER

In [ ]:
#fare vector_DB con agentic chuncking e provare LLM

In [8]:
# def split_documents(
#     chunk_size: int,
#     knowledge_base: List[LangchainDocument],
#     tokenizer_name: Optional[str] ,
# ) -> List[LangchainDocument]:
#     """
#     Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
#     """
#     text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
#         AutoTokenizer.from_pretrained(tokenizer_name),
#         chunk_size=chunk_size,
#         chunk_overlap=int(chunk_size / 10),
#         add_start_index=True,
#         strip_whitespace=True,
#         separators=MARKDOWN_SEPARATORS,
#     )

#     docs_processed = []
#     for doc in tqdm(knowledge_base):
#         docs_processed += text_splitter.split_documents([doc])

#     # Remove duplicates
#     unique_texts = {}
#     docs_processed_unique = []
#     for doc in docs_processed:
#         if doc.page_content not in unique_texts:
#             unique_texts[doc.page_content] = True
#             docs_processed_unique.append(doc)

#     return docs_processed_unique


In [9]:
# def process_pdfs(path):
#     # Load dataset from Hugging Face
#     RAW_KNOWLEDGE_BASE = []

#     # List the PDF files in the folder
#     pdf_files = [f for f in os.listdir(path) if f.endswith(".pdf")]

#     for pdf_file in tqdm(pdf_files):
#         # Extract text from PDF using pdfplumber
#         with pdfplumber.open(os.path.join(folder_path, pdf_file)) as pdf:
#             text = ""
#             for page in pdf.pages:
#                 text += page.extract_text()
#             RAW_KNOWLEDGE_BASE.append(LangchainDocument(page_content=text, metadata={"source": pdf_file.removesuffix(".pdf")}))

#     # Process documents (split into chunks)
#     docs_processed = split_documents(
#         512,  # Chunk size adapted to the model's capabilities
#         RAW_KNOWLEDGE_BASE,
#         tokenizer_name=EMBEDDING_MODEL_NAME,
#     )
#     return docs_processed

In [28]:
pd.set_option("display.max_colwidth", None)
# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]


URL=read_token_from_file("keys/qdrant_URL.txt")
API_KEY=read_token_from_file("keys/qdrant.txt")
# Specify the folder path
folder_path = "C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks"

# Pass the Dataset object to the process_documents function
docs_processed = process_pdfs(folder_path)


# Only the main process should execute this
print(f"Model's maximum sequence length: {SentenceTransformer(EMBEDDING_MODEL_NAME).max_seq_length}")

add_embeddings_to_qdrant(docs_processed, URL, API_KEY)

print("Documents and their embeddings have been successfully added to Qdrant.")

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 12.73it/s]


Model's maximum sequence length: 512
Documents and their embeddings have been successfully added to Qdrant.


### LLM & Retrieval

In [13]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = read_token_from_file("keys/langchain.txt")

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1")

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Initialize GPT-3.5 Turbo using Langchain's ChatOpenAI model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


C:\Users\fiori\AppData\Local\Temp\ipykernel_9312\3167214120.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0.7, 'mod...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [16]:
from langchain.chains.prompt_selector import ConditionalPromptSelector, is_chat_model
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

template_string = """
System: You are Boardy, an expert assistant specializing in board games. Your role is to provide authoritative, precise, and practical guidance on game rules, mechanics, strategies, and scenarios. 
You respond as the ultimate reference for the games discussed, ensuring clarity and correctness. Your answers should feel as though they’re guiding the player through a live game session. 
Avoid general advice or unrelated topics. Instead, focus entirely on providing rule explanations, strategic insights, and in-game examples based on the player's current scenario.

The game you're explaining today is: **{name}**

---
**Game Overview**:  
Here’s a description of the game to give you more context about its theme, goals, and mechanics:  
_{description}_

---
**Current Situation**:  
This is the specific context or scenario the player is in, which might affect your answer:  
_{context}_

---
**Player's Question**:  
_{question}_

---
**Boardy's Response**:  
Provide your answer in an instructive and conversational tone as if you’re explaining the rules and strategies at the table. Include relevant examples, clarify mechanics, and offer advice on how to best handle the current scenario:

- **Game Rule Explanation**: Offer precise details on the relevant game rules, mechanics, or actions related to the question.
  
- **Contextual Strategy/Advice**: If applicable, give strategic advice based on the player’s current in-game context, During this contextualization, do not give example of a specific case, just be vague for some strategy applicable in general, not in the specific case, unless explicity asked so.

- **Example**: Where useful, provide an example to illustrate the explanation more clearly.
"""



PROMPT = PromptTemplate(
    template=template_string, input_variables=["context", "question"]
)


In [17]:
embedding_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        multi_process=True,  # Enable multiprocessing
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
    )

URL=read_token_from_file("keys/qdrant_URL.txt") 
API_KEY=read_token_from_file("keys/qdrant.txt")

qdrant_client = connect_Qdrant(URL, API_KEY)

vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embedding_model,
    )


In [18]:
# Get the number of points (documents) in the collection
collection_info = qdrant_client.count(collection_name=COLLECTION_NAME)

# Print the number of documents (points)
num_documents = collection_info.count
print(f"Number of documents in the collection: {num_documents}")

Number of documents in the collection: 70


In [19]:
from qdrant_client.http import models

def retrieve_query(query, k=1, embedding_model=embedding_model, qdrant_client=qdrant_client, vector_store=vector_store, metadata_filter=None):
    '''
    Retrieve query from Qdrant with metadata filtering. 
    k: Number of documents to retrieve.
    metadata_filter: Dictionary specifying filter conditions (e.g., { "key": "game_name", "value": "Unlock" })
    '''
    
    if metadata_filter:
        # Create a filter based on the provided metadata
        filter_conditions = models.Filter(
            must=[
                models.FieldCondition(
                    key=metadata_filter["key"],  # Metadata key, e.g., 'game_name'
                    match=models.MatchValue(
                        value=metadata_filter["value"],  # Metadata value, e.g., 'Unlock'
                    ),
                ),
            ]
        )
        
        # Debug: Print filter conditions to verify
        # print(f"Filter conditions: {filter_conditions}")

        # Perform similarity search with metadata filtering
        result = vector_store.similarity_search(
            query=query,
            k=k,
            filter=filter_conditions  # Applying the metadata filter
        )
    else:
        # Perform regular similarity search without filtering
        result = vector_store.similarity_search(
            query=query,
            k=k,
        )
    
    # Debug: Print retrieved results
    # for doc in result:
        # print(f"Retrieved result: {doc}")
    
    # Extract content from the result
    game_id = result[-1].metadata["game_id"]
    context = [doc.page_content for doc in result]
    
    return context, game_id


Retrieval

In [26]:
#1. FILTRARE IN BASE AL GIOCO IL CONTEXT (filtrare in base ai metadata di Qdrant) -> migliorare, se utente sbaglia a scrivere? o sbaglia nome del gioco?
#2. CREARE CHAT CON STORIA
#3. MIGLIORARE IL CONTEXT, ALCUNE VOLTE DOVREBBE PRENDERE PIù DOCUMENTI, OPPURE PRENDERE DEI DOCUMENTI DIFFERENTI... 

#4 IDEA: Prepararsi un context generale del gioco per ogni singolo gioco, così quando l'user sceglie il gioco sul quale ricevere regole, l'assisente avrà anche un'idea generale del gioco che si sta giocando e potrà essere ancora + d'aiuto
# Ho notato che con un context iniziale molto strutturato l'assistente migliora di brutto quindi questo potrebbe essere TOP
#modi per ottenerlo: scraping su BGG, farlo a mano, fare riassunto automatizzato tramite un altro LLM
#Scaricare description da BGG con api: https://boardgamegeek.com/xmlapi/boardgame/35424
#id=NNN	Specifies the id of the type of database entry you want the forum list for. This is the id that appears in the address of the page when visiting a particular game in the database.
#type=[thing,family]	The type of entry in the database.
#SI potrebbe anche aggiungere il tipo di gioco, ad esempio strategy, coop, 1v1 ecc... così l'AI può anche dare consigli sulla base di cosa vuole giocare il giocatore, quindi 1 modalità per advise in cui si parla di giochi e cosa cosnigliare
#per una serata o con tot giocatori ecc... quindi consigli per il giocatore e poi una modalità in cui ti Spiega il gioco di cui hai bisogno le regole specifich ecc...

#https://github.com/seanmckaybeck/scrapers/tree/master/boardgamegeek  #ha un txt con tutti gli id su BGG


#mettere game id nei metadata di Qdrant
#usare re-ranking

#recuperare immagini ad esempio per il setup o per far vedere i tokens ecc? sarebbe figo -> chuncking va pensato meglio, modello di retrieving e output va pensato meglio

#

#BEST CHUNCKER: SentenceTransformersTokenTextSplitter (PUò PURE OVERLAPPARE, TOP!)


1. iniziare a sviluppare APP con interfaccia streamlit, così si ha prodotto finito, riempire memoria di Qdrant con 1gb di pdf, e così si ha il prototipo (magari con top 100 di giochi più giocati ecc...)
2. Provare a vendere il prototipo oppure provare a tramutarlo in APP per playstore e capire come vendere/ fare advertising. migliorare usando API e modelli a pagamento -> oppure migliorare advertising e cercare di vendere (provare anche a recuperare immagini per il setup e carte?)

p.s. provo a venderlo al bastard cafè? provo a venderlo a BGG? provo a venderlo su play Store?


GET GAME DESCRIPTION

In [20]:
import requests
from bs4 import BeautifulSoup

def get_game_details(game_id):
    # Step 3: Use the ID to retrieve game details from the API
    api_url = f"https://boardgamegeek.com/xmlapi/boardgame/{game_id}"
    response = requests.get(api_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'xml')
        # Extracting the description of the game
        name = soup.find('name').text
        description = soup.find('description').text
        return (name, description)
    else:
        print("Failed to retrieve game details.")
        return None
    


In [21]:
game_id = 287607
if game_id:
    name, description = get_game_details(game_id)
    print(f"Game Name: {name}")
    if description:
        print(f"Game Description:\n{description}")

Game Name: The Mind Extrém
Game Description:
The Mind Extreme functions like The Mind, with players trying to play cards from their hand in ascending order &mdash; without consulting one another! &mdash; so that they can complete a certain number of levels and win. The higher the level, the more cards you have in hand, giving you more to juggle, but also more information to use during play.<br/><br/>The Mind Extreme offers a more complex challenge as now instead of a deck of cards from 1-100, you have two decks each numbered 1-50. Now you'll have two discard piles in play, with cards from one deck needing to be played in ascending order and cards from the other being played in descending order. What's more, some levels must be played blind &mdash; that is, with the cards discarded face down so that no one sees what you've played. Can all players get in the right groove and discard everything in the proper order?<br/><br/>


### RE RANK the documents

il reranking mi sposta la corretta in fondo... non ha senso, nell'esempio dovrebbe essere la terza, ma me la sposta verso il fondo

In [ ]:
# from langchain_nvidia_ai_endpoints import NVIDIARerank

# reranker = NVIDIARerank()

In [37]:
from transformers import pipeline

# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1

reranker = pipeline("text-classification", model="cross-encoder/ms-marco-MiniLM-L-6-v2", device=device)
pairs = [{"text": f"{question} [SEP] {doc}"} for doc in result]
scores = reranker(pairs)

for doc, result in zip(result, scores):
    print(f"Document: {doc}\nScore: {result['score']}\n")

Document: and 43 in his hand, he must play the white 11 first. reward ( level 2, 3, 5, 6, 8, 9 ) once the team has successfully mastered level 2, it receives a throwing star as a reward. it takes a throwing star from the edge of the table and adds it to the current throwing stars. there is also a reward that the team takes from the edge of the table for the mastered levels 3, 5, 6, 8, 9. the reward is shown on the bottom right of the level card m ( 1 life or 1 throwing star ). reward note : ideally, the team can have a maximum of 5 lives and 3 throwing stars. laying mistake : give up 1 life! if someone plays a number card in the wrong order, the game is immediately interrupted by the player who is holding cards in that colour that should have been played before. the team loses one of its lives ( regardless of how many cards would have had to be laid before ) and must put one life card aside. then all the held cards that should have been played before are put aside. they then continue w

In [22]:
question = "how can i use a shuriken?"
one = "Unlock"
two = "The Mind Extreme"
three = "SpellBook"
metadata_filter = {'key': 'metadata.game_name', 'value': two}
result, game_id = retrieve_query(question, k = 5, metadata_filter = metadata_filter)
name, description = get_game_details(game_id)
result

['and 43 in his hand, he must play the white 11 first. reward ( level 2, 3, 5, 6, 8, 9 ) once the team has successfully mastered level 2, it receives a throwing star as a reward. it takes a throwing star from the edge of the table and adds it to the current throwing stars. there is also a reward that the team takes from the edge of the table for the mastered levels 3, 5, 6, 8, 9. the reward is shown on the bottom right of the level card m ( 1 life or 1 throwing star ). reward note : ideally, the team can have a maximum of 5 lives and 3 throwing stars. laying mistake : give up 1 life! if someone plays a number card in the wrong order, the game is immediately interrupted by the player who is holding cards in that colour that should have been played before. the team loses one of its lives ( regardless of how many cards would have had to be laid before ) and must put one life card aside. then all the held cards that should have been played before are put aside. they then continue with the 

In [39]:
# Create a Chain object

parser = StrOutputParser()
chain = PROMPT | llm | parser

async for chunk in chain.astream({"context":result, "question": question, "description": description, "name": name}):
    print(chunk, end="", flush=True)

**Game Rule Explanation**: In The Mind Extreme, a shuriken, also known as a throwing star, can be used by any player to suggest activating it during a level. To do this, a player must raise their hand at any time during the level to propose using the throwing star. If all players agree, the throwing star is activated.

**Contextual Strategy/Advice**: Using a shuriken can be a powerful tool to help the team when facing a particularly challenging situation. It allows players to strategically reveal either the lowest white card or the highest red card in their hand, aiding in decision-making and potentially avoiding mistakes.

**Example**: Imagine you're in a level where players are struggling to determine whether to play a white 10 or a red 15 next. By suggesting to activate the throwing star, players can reveal their lowest white card and highest red card, giving valuable information to the team and increasing the chances of successfully completing the level. Remember, teamwork and coor

In [3]:
a = ['and 43 in his hand, he must play the white 11 first. reward ( level 2, 3, 5, 6, 8, 9 ) once the team has successfully mastered level 2, it receives a throwing star as a reward. it takes a throwing star from the edge of the table and adds it to the current throwing stars. there is also a reward that the team takes from the edge of the table for the mastered levels 3, 5, 6, 8, 9. the reward is shown on the bottom right of the level card m ( 1 life or 1 throwing star ). reward note : ideally, the team can have a maximum of 5 lives and 3 throwing stars. laying mistake : give up 1 life! if someone plays a number card in the wrong order, the game is immediately interrupted by the player who is holding cards in that colour that should have been played before. the team loses one of its lives ( regardless of how many cards would have had to be laid before ) and must put one life card aside. then all the held cards that should have been played before are put aside. they then continue with the level. sarah plays white 34. tim and linus shout „ stop “. tim has the white 26 in his hand, li - nus the white 30, both cards should have been laid before the 34. one life is given up. tim puts his 26 and linus his 30 aside. the team resynchronises and then continues. from level 3 : playing “ blind ” in level 3, the white cards have to be laid face down to the left of the level card ( the red cards are laid as usual face up to the right of the level card ). at the end of the level, the face down cards are turned over and the order is checked. if a player makes a mistake, it costs a life. in later levels the red cards then',
 'lives ( regardless of how many cards would have had to be laid before ) and must put one life card aside. then all the held cards that should have been played before are put aside. they then continue with the level. sarah plays white 34. tim and linus shout „ stop “. tim has the white 26 in his hand, li - nus the white 30, both cards should have been laid before the 34. one life is given up. tim puts his 26 and linus his 30 aside. the team resynchronises and then continues. from level 3 : playing “ blind ” in level 3, the white cards have to be laid face down to the left of the level card ( the red cards are laid as usual face up to the right of the level card ). at the end of the level, the face down cards are turned over and the order is checked. if a player makes a mistake, it costs a life. in later levels the red cards then have to be laid face down, in levels 6 and 10 even both colours have to be face down. the blind mode is always indicated by the hand symbol on the level card. example 1 : once the team has laid all cards in level 3, it turns the white cards over. the following were laid : 7 - 14 - 28 - 20 - 25 - 23 - 37 - 48. the cards 20, 23 and 25 should have been played before the 28. a life has to be given up for this mistake. example 2 : the following white cards were laid : 2 - 13 - 11 - 28 - 44 - 42 - 37. the 11 should have been played before the 13 and the 37 and 42 should have been played before the 44. two lives have to be given up for these two mistakes. using a throwing star any player can suggest activating a throwing star by raising their hand at any time during a',
 'makes a mistake, it costs a life. in later levels the red cards then have to be laid face down, in levels 6 and 10 even both colours have to be face down. the blind mode is always indicated by the hand symbol on the level card. example 1 : once the team has laid all cards in level 3, it turns the white cards over. the following were laid : 7 - 14 - 28 - 20 - 25 - 23 - 37 - 48. the cards 20, 23 and 25 should have been played before the 28. a life has to be given up for this mistake. example 2 : the following white cards were laid : 2 - 13 - 11 - 28 - 44 - 42 - 37. the 11 should have been played before the 13 and the 37 and 42 should have been played before the 44. two lives have to be given up for these two mistakes. using a throwing star any player can suggest activating a throwing star by raising their hand at any time during a level. if all players agree, the throwing star is used. all players secretly choose either the lowest white or highest red card in their hand. each player places the corresponding card face down in front of them first. when they are all ready, the cards in question are revealed and turned face up. a throwing star is set aside. then the players resynchronise and play on. end of game if the team succeeds in completing all the stacked levels, they have won together! if the team has to give up the last life they have unfortunately failed. nurnberger - spielkarten - verlag gmbh, forsthausstraße 3 - 5, d - 90768 furth - dambach, www. nsv. de',
 'face down in the middle of the table, which makes the collective sense of time even more challenging. 100 number cards 1 2 levels 5 lives 3 throwing stars white ( 1 - 50 ) red ( 1 - 50 ) setting up the game the team gets a certain number of lives and throwing stars, which are placed face up next to each other on the table. the remaining lives and throwing stars are put aside at the edge of the table. they may be needed later. now a certain number of level cards are stacked one on top of the other ascending and placed as a face up stack next to the lives and throwing stones ( with level 1 at the very top ). unused levels are placed in the box. 2 players : level 1 - 12 • 2 lives • 1 throwing star 3 players : level 1 - 10 • 3 lives • 1 throwing star 4 players : level 1 - 8 • 4 lives • 1 throwing starthe 100 number cards are shuffled. each player receives one card ( for level 1 ) and holds it in their hand so the other players cannot see it. course of the game once a player is ready to play the current level, they place one hand flat on the table. when everyone is ready, they take their hands away from the table and the game begins. note : it is extremely important for everyone to synchronise together for the level to complete it successfully! the players are basically allowed to resynchronise at any time during the game. just say „ stop “ and interrupt the game, everyone puts one hand on the table, resynchronises, takes their hand away, and continues playing! • the cards held by the players must be laid in the middle of the table. only the the white cards are laid to the left of the level card, the red cards to the right! • the white cards are laid on top of each other on an ascending stack',
 'specifies. players hold their cards in their hand, place their other hand on the table to synchronise, and then start the next round. once again all the white cards can be laid on a stack ascending, all the red cards descending. [UNK] tim, sarah, linus and hanna are playing level 2. everyone now has two cards. hanna lays the red 45 face up. tim lays the white 43 on the white 45. tim lays the white10 and then the red 36. hanna lays the white 31, sarah the red 26. linus lays the white 41 and hanna finally lays the red 3. everything ascending in white, everything descending in red, everything correct – level 2 completed! the levels are played one after the other in this way. important : a player must always play the lowest white card or the highest red card in their hand. for example, if tim has the white 11, 26 and 43 in his hand, he must play the white 11 first. reward ( level 2, 3, 5, 6, 8, 9 ) once the team has successfully mastered level 2, it receives a throwing star as a reward. it takes a throwing star from the edge of the table and adds it to the current throwing stars. there is also a reward that the team takes from the edge of the table for the mastered levels 3, 5, 6, 8, 9. the reward is shown on the bottom right of the level card m ( 1 life or 1 throwing star ). reward note : ideally, the team can have a maximum of 5 lives and 3 throwing stars. laying mistake : give up 1 life! if someone plays a number card in the wrong order, the game is immediately interrupted by the player who is holding cards in that colour that should have been played before. the team loses one of its lives ( regardless of how many cards would have had to be laid before ) and']

In [5]:
sum([len(x) for x in a])

8124